## Installations

In [1]:
# ! pip install dspy-ai==2.4.3
! pip install structlog
# ! pip install qdrant-client==1.8.0
# ! pip install fastembed
! pip install Jinja2
! pip install google-generativeai

  Using cached structlog-24.1.0-py3-none-any.whl.metadata (6.9 kB)
Using cached structlog-24.1.0-py3-none-any.whl (65 kB)
  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)
  Using cached google_generativeai-0.5.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.1-py3-none-any.whl.metadata (5.2 kB)
  Using cached google_api_core-2.18.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_api_python_client-2.125.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached google_auth-2.29.0-py2.py3-none-any.whl.metadata (4.7 kB)
  Using cached protobuf-5.26.1-cp37-abi3-macosx_10_9_universal2.whl.metadata (592 bytes)
  Using cached proto_plus-1.23.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached cachetools-5.3.3-py3-n

In [17]:
! pip install Jinja2


  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)


In [1]:
!pip show structlog

Name: structlog
Version: 24.1.0
Summary: Structured Logging for Python
Home-page: 
Author: 
Author-email: Hynek Schlawack <hs@ox.cx>
License: 
Location: /home/tanmay/miniconda3/envs/dspy_fork/lib/python3.9/site-packages
Requires: 
Required-by: 


In [58]:
# ! pip install Jinja2

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached Jinja2-3.1.3-py3-none-any.whl.metadata (3.3 kB)
Using cached Jinja2-3.1.3-py3-none-any.whl (133 kB)


## Set LLM API key

In [2]:
# import openai
# openai.api_key = "sk-foobar"

import os


# Or add your OPEN_AI_API_KEY
os.environ['OPENAI_API_KEY'] = 'SOME KEY'

if 'OPENAI_API_KEY' not in os.environ:
    raise Exception('Environment variable "OPENAI_API_KEY" not set')


In [ ]:
import os
# os.environ.pop('OPENAI_API_KEY')
os.environ.get('OPENAI_API_KEY')

In [4]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)

## Import FAQs from the FAQs markdown

In [5]:
# Load FAQs
import re
import random

f = open("faqs_parsings.md")
markdown_content = f.read()

def parse_questions(markdown_content):
    # Regular expression pattern for finding questions
    question_pattern = r'Q: (.+?)\nA: (.+?)(?=\n\nQ:|\Z)'

    matches = re.findall(question_pattern, markdown_content, re.DOTALL)
    ret_l = []
    for match in matches:
        ret_l.append({"input_address": match[0], "address_components": match[1], })
    return ret_l

# Parsing the markdown content to get only questions
all_faqs = parse_questions(markdown_content)

random.seed(42)
random.shuffle(all_faqs)

# Displaying the first few extracted questions
all_faqs[:5]  # Displaying only the first few for brevity


[{'input_address': 'chandra nagar thakur mohlla koteswar mandir Gwalior Gwalior Madhya Pradesh 474003',
  'address_components': '{ "house_number": "N/A", "city": "Gwalior", "state": "Madhya Pradesh", "pincode": "474003", "locality": "chandra nagar thakur mohlla", "landmark": "koteswar mandir"}'},
 {'input_address': 'WARD 09 LASANPUR BUARI DAGARUA PURNIA',
  'address_components': '{"house_number": "N/A", "ward_number": "09", "city": "Purnia", "state": "N/A", "pincode": "N/A", "locality": "Lasanpur Buari Dagarua"}'},
 {'input_address': 'Room No 133 RAK Road  Wadala Arvi Mumbai Maharashtra 400031',
  'address_components': '{ "house_number": "Room No 133", "city": "Mumbai", "state": "Maharashtra", "pincode": "400031", "locality": "Wadala", "street_name": "RAK Road"}'},
 {'input_address': 'C-102 Payal Colony Sector 14 Gurgaon',
  'address_components': '{ "house_number": "C-102", "city": "Gurgaon", "state": "Haryana", "pincode": "N/A", "colony_name": "Payal Colony", "sector": "Sector 14" } '

## Initalize the LLM and retriever models and configure them in DSPy

In [6]:
# NOTE: assumes weaviate-client version 3.*

# Connect to Weaviate Retriever and configure LLM
import dspy
# import os

import openai


gemini = dspy.Google(api_key="SOME KEY")
turbo_35 = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=1000, model_type='chat')
turbo_4 = dspy.OpenAI(model="gpt-4-turbo", max_tokens=1000, model_type='chat')


dspy.settings.configure(lm=turbo_35)

/home/tanmay/miniconda3/envs/dspy_fork/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Test that the LLM is working fine

In [7]:
# print(dspy.settings.lm("Write a 3 line poem about neural networks, as if you are from New Delhi, India"))

# context_example = dspy.OpenAI(model="gpt-4")
# with dspy.context(llm=context_example):
#     print(context_example("Write a 3 line poem about neural networks."))

In [8]:
# reply = turbo_4("")
# print(reply[0])

## Wrap each FAQ into an `dspy.Example` object

The dspy `Example` object optionally lets you attach metadata, or additional labels, to input/output pairs.

For example, you may want to jointly supervise the answer as well as the context the retrieval system produced to feed into the answer generator.

In [9]:
len(all_faqs)

21

In [10]:
# Load into dspy datasets
import dspy

trainset = all_faqs[:9] # 20 examples for training
devset = all_faqs[9:] # remaining examples for development
# testset = all_faqs[14:] # 14 examples for testing


trainset = [dspy.Example(**faq).with_inputs("input_address") for faq in trainset]
devset = [dspy.Example(**faq).with_inputs("input_address") for faq in devset]
# testset = [dspy.Example(**faq).with_inputs("question") for faq in testset]

In [11]:
trainset[0]

Example({'input_address': 'chandra nagar thakur mohlla koteswar mandir Gwalior Gwalior Madhya Pradesh 474003', 'address_components': '{ "house_number": "N/A", "city": "Gwalior", "state": "Madhya Pradesh", "pincode": "474003", "locality": "chandra nagar thakur mohlla", "landmark": "koteswar mandir"}'}) (input_keys={'input_address'})

# 2. LLM Metrics

Define a Metric for Performance.

In [19]:
import json
# This is a WIP, the next step is to optimize this metric as itself a DSPy module (pretty meta)

# Reference - https://github.com/stanfordnlp/dspy/blob/main/examples/tweets/tweet_metric.py

metricLM = turbo_4

# Signature for LLM assessments.

class Assessment1(dspy.Signature):
    """Assess the quality of an input address parsed into address components based on a criterion."""

    assessment_criterion = dspy.InputField(desc="The evaluation criterion.")
    assessed_input_address = dspy.InputField(desc="The input address.")
    assessed_components = dspy.InputField(desc="The components from the input address.")
    assessment_answer = dspy.OutputField(desc="A rating between 1 and 5. Only output the rating and nothing else.")

def metric1(gold, pred, trace=None):
    input_address = gold.input_address
    predicted_components = pred.address_components.lower()
    gold_components = gold.address_components.lower()

    
    print(f"Input address: {input_address}")
    print(f"Predicted components: {predicted_components}")

    
    correctness = "We want each parsed component (i.e, each key-value pair) should be correct on its own."
    information_repetition = "We do not want the same piece of information from the input address being repeated into multiple components."
    information_completeness = "We do not want some critical piece of information to be missed in the parsed components."
    irrelevance_filter = "We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components."

    try:
        j = json.loads(predicted_components)
    except:
        # no tolerance for non-parseable output
        if not trace: return False
        else: return 0.0
    
    # make sure the keys match.
    pred_keys = set(json.loads(predicted_components).keys())
    gold_keys = set(json.loads(gold_components).keys())
    total_keys = len(pred_keys.union(gold_keys))
    num_keys_match = len(pred_keys.intersection(gold_keys))

    if num_keys_match == 0:
        if not trace: return False
        else: return 0.0
    ratio_keys_match = num_keys_match / total_keys

    with dspy.context(lm=metricLM):
        # context = dspy.Retrieve(k=3)(question).passages
        correctness = dspy.ChainOfThought(Assessment1)(assessment_criterion=correctness, assessed_input_address=input_address, assessed_components=predicted_components)
        information_repetition = dspy.ChainOfThought(Assessment1)(assessment_criterion=information_repetition, assessed_input_address=input_address, assessed_components=predicted_components)
        information_completeness = dspy.ChainOfThought(Assessment1)(assessment_criterion=information_completeness, assessed_input_address=input_address, assessed_components=predicted_components)
        irrelevance_filter = dspy.ChainOfThought(Assessment1)(assessment_criterion=irrelevance_filter, assessed_input_address=input_address, assessed_components=predicted_components)
    print(f"correctness: {correctness.assessment_answer}")
    print(f"information_repetition: {information_repetition.assessment_answer}")
    print(f"information_completeness: {information_completeness.assessment_answer}")
    print(f"irrelevance_filter: {irrelevance_filter.assessment_answer}")
    print("><><><><><><><><><><<><><><><><><><><><><><><")
    print("history of metricLM:","\n\n",metricLM.inspect_history(n=3))
    print("><><><><><><><><><><<><><><><><><><><><><><><")

    a1 = float(information_repetition.assessment_answer)
    a2 = float(information_completeness.assessment_answer)
    a3 = float(information_repetition.assessment_answer)
    a4 = float(correctness.assessment_answer)
    if not trace:
        return (a1 + a2 + a3 + a4 > 16) and (ratio_keys_match > 0.8)
    else:
        return (a1 + a2*2 + a3 + ratio_keys_match*5) / 25.0


# 3. The DSPy Programming Model

This block of first code will initilaize the `GenerateAnswer` signature.

Then we will compose a `dspy.Module` consisting of:
- Retrieve
- GenerateAnswer

The DSPy programming model is one of the most powerful aspects of DSPy, we get:
- An intuitive interface to compose prompts into programs.
- A clean way to organize prompts into Signatures.
- Structured output parsing with `dspy.OutputField`
- Built-in prompt extensions such as `ChainOfThought`, `ReAct`, and more!

In [27]:
class GenerateComponents(dspy.Signature):
    """You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.
    """
    
    input_address = dspy.InputField()
    address_components = dspy.OutputField()

class ValidateComponents(dspy.Signature):
    """You are validating (and correcting) an input address (in India) to address components mapping. Your job is to correct the parsings."""
    
    input_address = dspy.InputField()
    input_address_components = dspy.InputField()
    address_components = dspy.OutputField(desc="validated and corrected address components")


In [28]:
class AddressParser(dspy.Module):
    def __init__(self):
        super().__init__()

        self.generate_components = dspy.ChainOfThought(GenerateComponents)
        self.validate_components = dspy.ChainOfThought(ValidateComponents)
    
    def forward(self, input_address):
        history = []
        predictions = []
        prediction = self.generate_components(input_address=input_address)
        history.append(dspy.settings.lm.inspect_history(n=1))
        predictions.append(prediction.address_components)

        # prediction = self.validate_components(input_address=input_address, input_address_components=prediction.address_components)
        # history.append(dspy.settings.lm.inspect_history(n=1))
        # predictions.append(prediction.address_components)

        address_components = prediction.address_components

        return dspy.Prediction(address_components=address_components, input_address=input_address, predictions=predictions, history=history)

In [29]:
uncompiled_bot = AddressParser()
# compiled_bot = AddressParser()

# 4. DSPy Optimization

In [30]:
from dspy.evaluate.evaluate import Evaluate

evaluate = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

evaluate(uncompiled_bot, metric=metric1)




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607
Re

correctness: 5
information_repetition: 5
information_completeness: 4
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607

Assessed Components: { "building name": "sumi bhavan", "locality": "alanthara", "post office": "venjaramoodu", "city": "trivandrum", "state": "ke

correctness: 5
information_repetition: 5
information_completeness: 5
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: Hno 403, Domlur, 2nd Flr, Indiranagar, Bengaluru

Assessed Components: { "house number": "403", "locality": "domlur", "floor": "2nd", "area": "indiranagar", "city": "bengaluru" }

Reasoning: Let's think step by step in order to pr

correctness: 5
information_repetition: 3
information_completeness: 5
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: 86 SHTRAPAL SOSAYTE BAMROLI Surat Surat Gujarat 394221

Assessed Components: { "house number": "86", "street name": "shtrapal sosayte bamroli", "locality": "surat", "city": "surat", "state": "gujarat", "pin code": "394221" }

Reas




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: HNO 8146 STNO 13 MILERGANJ
Reasoning: Let's think step by step in order to produce the

...



Average Metric: 0 / 12  (0.0): 100%|██████████| 12/12 [05:37<00:00, 28.10s/it]

correctness: 3
information_repetition: 5
information_completeness: 3
irrelevance_filter: 4
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: HNo 136 Thane Mumbai Maharashtra

Assessed Components: { "house number": "136", "street name": "thane", "city": "mumbai", "state": "maharashtra" }

Reasoning: Let's think step by step in order to produce the assessment_answer. We 


/home/tanmay/work/dspy_fork/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,example_input_address,example_address_components,pred_address_components,pred_input_address,predictions,history,metric1
0,"Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607","{""house_number"": ""N/A"", ""house_name"": ""Sumi Bhavan"", ""locality"": ""Alanthara"", ""city"": ""Trivandrum"", ""state"": ""Kerala"", ""pincode"": ""695607"", ""post_office"": ""Venjaramoodu P O"", ""village"": ""nellanad"", ""district"": ""Trivandrum""}","{ ""Building Name"": ""Sumi Bhavan"", ""Locality"": ""Alanthara"", ""Post Office"": ""Venjaramoodu"", ""City"": ""Trivandrum"", ""State"": ""Kerala"", ""Pin Code"": ""695607"" }","Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607","['{\n ""Building Name"": ""Sumi Bhavan"",\n ""Locality"": ""Alanthara"",\n ""Post Office"": ""Venjaramoodu"",\n ""City"": ""Trivandrum"",\n ""State"": ""Kerala"",\n ""Pin Code"": ""695607""\n}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
1,"Hno 403, Domlur, 2nd Flr, Indiranagar, Bengaluru","{ ""house_number"": ""403"", ""city"": ""Bengaluru"", ""state"": ""Karnataka"", ""pincode"": ""N/A"", ""locality"": ""Domlur"", ""floor"": ""2nd Flr"", ""area"": ""Indiranagar"" }","{ ""House Number"": ""403"", ""Locality"": ""Domlur"", ""Floor"": ""2nd"", ""Area"": ""Indiranagar"", ""City"": ""Bengaluru"" }","Hno 403, Domlur, 2nd Flr, Indiranagar, Bengaluru","['{\n ""House Number"": ""403"",\n ""Locality"": ""Domlur"",\n ""Floor"": ""2nd"",\n ""Area"": ""Indiranagar"",\n ""City"": ""Bengaluru""\n}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
2,86 SHTRAPAL SOSAYTE BAMROLI Surat Surat Gujarat 394221,"{ ""house_number"": ""86"", ""city"": ""Surat"", ""state"": ""Gujarat"", ""pincode"": ""394221"", ""society_name"": ""SHTRAPAL SOSAYTE"", ""locality"": ""BAMROLI"" }","{ ""House Number"": ""86"", ""Street Name"": ""SHTRAPAL SOSAYTE BAMROLI"", ""Locality"": ""Surat"", ""City"": ""Surat"", ""State"": ""Gujarat"", ""Pin Code"": ""394221"" }",86 SHTRAPAL SOSAYTE BAMROLI Surat Surat Gujarat 394221,"['{\n ""House Number"": ""86"",\n ""Street Name"": ""SHTRAPAL SOSAYTE BAMROLI"",\n ""Locality"": ""Surat"",\n ""City"": ""Surat"",\n ""State"": ""Gujarat"",\n ""Pin Code"": ""394221""\n}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
3,HNO 8146 STNO 13 MILERGANJ,"{""house_number"": ""8146"", ""streen_number"": ""13"", ""city"": ""MILERGANJ"", ""state"": ""N/A"", ""pincode"": ""N/A""}","{ ""House Number"": ""8146"", ""Street Number"": ""13"", ""Locality"": ""MILERGANJ"" }",HNO 8146 STNO 13 MILERGANJ,"['{\n ""House Number"": ""8146"",\n ""Street Number"": ""13"",\n ""Locality"": ""MILERGANJ""\n}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
4,"HSE NO 12/231, 21 RAM NAGAR Ludhiana Ludhiana Punjab 141003","{""house_number"": ""12/231"", ""city"": ""Ludhiana"", ""state"": ""Punjab"", ""pincode"": ""141003"", ""locality"": ""RAM NAGAR""}","{ ""House Number"": ""12/231"", ""Street Name"": ""Ram Nagar"", ""City"": ""Ludhiana"", ""State"": ""Punjab"", ""Pincode"": ""141003"" }","HSE NO 12/231, 21 RAM NAGAR Ludhiana Ludhiana Punjab 141003","['{\n ""House Number"": ""12/231"",\n ""Street Name"": ""Ram Nagar"",\n ""City"": ""Ludhiana"",\n ""State"": ""Punjab"",\n ""Pincode"": ""141003""\n}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False


0.0

# BootstrapFewShot

In [31]:
from dspy.teleprompt import BootstrapFewShot

teleprompter = BootstrapFewShot(metric=metric1, max_labeled_demos=8, max_rounds=3)

# also common to init here, e.g. Rag()
compiled_bot = teleprompter.compile(uncompiled_bot, trainset=devset)




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: 3 d/o mohd hasan, jeevangarh, hn 96, ambari Jeevangarh jeevangarh, Jiwangarh, ambari,V

correctness: 5
information_repetition: 5
information_completeness: 4
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607

Assessed Components: {"house_name": "sumi bhavan", "locality": "alanthara", "post_office": "venjaramoodu p o", "city": "trivandrum", "state": "ke

correctness: 5
information_repetition: 5
information_completeness: 3
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: Hno 403, Domlur, 2nd Flr, Indiranagar, Bengaluru

Assessed Components: {"house_number": "403", "locality": "domlur", "floor": "2nd", "city": "bengaluru", "state": "n/a", "pincode": "n/a"}

Reasoning: Let's think step by step in or

correctness: 5
information_repetition: 5
information_completeness: 4
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: 86 SHTRAPAL SOSAYTE BAMROLI Surat Surat Gujarat 394221

Assessed Components: {"house_number": "86", "street_name": "shtrapal sosayte bamroli", "city": "surat", "state": "gujarat", "pincode": "394221"}

Reasoning: Let's think step 


 33%|███▎      | 4/12 [03:32<07:05, 53.15s/it]


correctness: 5
information_repetition: 5
information_completeness: 2
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: HNO 8146 STNO 13 MILERGANJ

Assessed Components: {"house_number": "8146", "street_number": "13", "locality": "milerganj", "city": "n/a", "state": "n/a", "pincode": "n/a"}

Reasoning: Let's think step by step in order to produce th


  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Bootstrapped 4 full traces after 1 examples in round 2.


### Inspect the compiled prompt

In [34]:
# the trainset and devset have been exchanged
trainset[0]

Example({'input_address': 'chandra nagar thakur mohlla koteswar mandir Gwalior Gwalior Madhya Pradesh 474003', 'address_components': '{ "house_number": "N/A", "city": "Gwalior", "state": "Madhya Pradesh", "pincode": "474003", "locality": "chandra nagar thakur mohlla", "landmark": "koteswar mandir"}'}) (input_keys={'input_address'})

In [36]:
for gold in trainset:
    print(gold.input_address)

chandra nagar thakur mohlla koteswar mandir Gwalior Gwalior Madhya Pradesh 474003
WARD 09 LASANPUR BUARI DAGARUA PURNIA
Room No 133 RAK Road  Wadala Arvi Mumbai Maharashtra 400031
C-102 Payal Colony Sector 14 Gurgaon
D-4/181,182 D-BLOCK SULTANPURI New Delhi Delhi Delhi 110086
JALAL ROAD SAMADH BHAI Baghapurana Moga Punjab 142057
JALALABAD  Dharakot Moga Punjab 142042
SAHYADRI HOSPITAL LTD HADAPSAR PUNE  BHOSALE GARDEN HADAPSAR  Pune Pune Maharashtra 411028
House 123 Near Ram Mandir Kakrala Punjab


In [37]:
gold = trainset[2]
pred = compiled_bot(gold.input_address)




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607
Re

In [39]:
print(gold.address_components)
print(pred.address_components)

{ "house_number": "Room No 133", "city": "Mumbai", "state": "Maharashtra", "pincode": "400031", "locality": "Wadala", "street_name": "RAK Road"}
{"room_number": "133", "street_name": "RAK Road", "locality": "Wadala Arvi", "city": "Mumbai", "state": "Maharashtra", "pincode": "400031"}


In [38]:

metric1(gold=gold, pred=pred, trace=None)

Input address: Room No 133 RAK Road  Wadala Arvi Mumbai Maharashtra 400031
Predicted components: {"room_number": "133", "street_name": "rak road", "locality": "wadala arvi", "city": "mumbai", "state": "maharashtra", "pincode": "400031"}
correctness: 5
information_repetition: 5
information_completeness: 5
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Ad

False

In [91]:
gold.address_components

'{ "house_number": "86", "city": "Surat", "state": "Gujarat", "pincode": "394221", "society_name": "SHTRAPAL SOSAYTE", "locality": "BAMROLI" }'

In [ ]:
llm.inspect_history(n=1)

### Evaluate the Compiled Bot

In [40]:
evaluate(compiled_bot, metric=metric1)




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607
Re




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607
Re




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607
Re




You are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component type and the value as the component value. Your performance will be evaluated on the following criteria:
    1. information_repetition: same information from the input address should not be repeated in the components.
    2. information_completeness: a critical address information should not be missed from the components.
    3. irrelevance_filter: irrelevant piece of information (eg- spurious digits, characters etc) should be filtered out.
    4. correctness: each parsed component (i.e, each key-value pair) should be correct on its own.

---

Follow the following format.

Input Address: ${input_address}
Reasoning: Let's think step by step in order to ${produce the address_components}. We ...
Address Components: ${address_components}

---

Input Address: Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607
Re

correctness: 4
information_repetition: 5
information_completeness: 5
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: HSE NO 12/231, 21 RAM NAGAR Ludhiana Ludhiana Punjab 141003

Assessed Components: {"house_number": "12/231", "street_name": "ram nagar", "city": "ludhiana", "state": "punjab", "pincode": "141003"}

Reasoning: Let's think step by s

...



Average Metric: 5 / 12  (41.7): 100%|██████████| 12/12 [08:23<00:00, 42.00s/it]

correctness: 4
information_repetition: 5
information_completeness: 3
irrelevance_filter: 5
><><><><><><><><><><<><><><><><><><><><><><><



Assess the quality of an input address parsed into address components based on a criterion.

---

Follow the following format.

Assessment Criterion: The evaluation criterion.

Assessed Input Address: The input address.

Assessed Components: The components from the input address.

Reasoning: Let's think step by step in order to ${produce the assessment_answer}. We ...

Assessment Answer: A rating between 1 and 5. Only output the rating and nothing else.

---

Assessment Criterion: We do not want some irrelevant piece of information (eg- spurious digits, etc) to be captured in unrelated components.

Assessed Input Address: HNo 136 Thane Mumbai Maharashtra

Assessed Components: {"house_number": "136", "city": "mumbai", "state": "maharashtra", "pincode": "n/a"}

Reasoning: Let's think step by step in order to produce the assessment_answer. We start by


/home/tanmay/work/dspy_fork/dspy/evaluate/evaluate.py:266: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' '✔️ [True]' 'False' 'False' 'False']' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,example_input_address,example_address_components,pred_address_components,pred_input_address,predictions,history,metric1
0,"Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607","{""house_number"": ""N/A"", ""house_name"": ""Sumi Bhavan"", ""locality"": ""Alanthara"", ""city"": ""Trivandrum"", ""state"": ""Kerala"", ""pincode"": ""695607"", ""post_office"": ""Venjaramoodu P O"", ""village"": ""nellanad"", ""district"": ""Trivandrum""}","{""house_name"": ""Sumi Bhavan"", ""locality"": ""Alanthara"", ""post_office"": ""Venjaramoodu P O"", ""city"": ""Trivandrum"", ""state"": ""Kerala"", ""pincode"": ""695607""}","Sumi Bhavan Alanthara Venjaramoodu P O,nellanad Trivandrum Trivandrum Kerala 695607","['{""house_name"": ""Sumi Bhavan"", ""locality"": ""Alanthara"", ""post_office"": ""Venjaramoodu P O"", ""city"": ""Trivandrum"", ""state"": ""Kerala"", ""pincode"": ""695607""}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
1,"Hno 403, Domlur, 2nd Flr, Indiranagar, Bengaluru","{ ""house_number"": ""403"", ""city"": ""Bengaluru"", ""state"": ""Karnataka"", ""pincode"": ""N/A"", ""locality"": ""Domlur"", ""floor"": ""2nd Flr"", ""area"": ""Indiranagar"" }","{""house_number"": ""403"", ""locality"": ""Domlur"", ""floor"": ""2nd"", ""city"": ""Bengaluru"", ""state"": ""N/A"", ""pincode"": ""N/A""}","Hno 403, Domlur, 2nd Flr, Indiranagar, Bengaluru","['{""house_number"": ""403"", ""locality"": ""Domlur"", ""floor"": ""2nd"", ""city"": ""Bengaluru"", ""state"": ""N/A"", ""pincode"": ""N/A""}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",✔️ [True]
2,86 SHTRAPAL SOSAYTE BAMROLI Surat Surat Gujarat 394221,"{ ""house_number"": ""86"", ""city"": ""Surat"", ""state"": ""Gujarat"", ""pincode"": ""394221"", ""society_name"": ""SHTRAPAL SOSAYTE"", ""locality"": ""BAMROLI"" }","{""house_number"": ""86"", ""street_name"": ""SHTRAPAL SOSAYTE BAMROLI"", ""city"": ""Surat"", ""state"": ""Gujarat"", ""pincode"": ""394221""}",86 SHTRAPAL SOSAYTE BAMROLI Surat Surat Gujarat 394221,"['{""house_number"": ""86"", ""street_name"": ""SHTRAPAL SOSAYTE BAMROLI"", ""city"": ""Surat"", ""state"": ""Gujarat"", ""pincode"": ""394221""}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
3,HNO 8146 STNO 13 MILERGANJ,"{""house_number"": ""8146"", ""streen_number"": ""13"", ""city"": ""MILERGANJ"", ""state"": ""N/A"", ""pincode"": ""N/A""}","{""house_number"": ""8146"", ""street_number"": ""13"", ""locality"": ""MILERGANJ"", ""city"": ""N/A"", ""state"": ""N/A"", ""pincode"": ""N/A""}",HNO 8146 STNO 13 MILERGANJ,"['{""house_number"": ""8146"", ""street_number"": ""13"", ""locality"": ""MILERGANJ"", ""city"": ""N/A"", ""state"": ""N/A"", ""pincode"": ""N/A""}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False
4,"HSE NO 12/231, 21 RAM NAGAR Ludhiana Ludhiana Punjab 141003","{""house_number"": ""12/231"", ""city"": ""Ludhiana"", ""state"": ""Punjab"", ""pincode"": ""141003"", ""locality"": ""RAM NAGAR""}","{""house_number"": ""12/231"", ""street_name"": ""RAM NAGAR"", ""city"": ""Ludhiana"", ""state"": ""Punjab"", ""pincode"": ""141003""}","HSE NO 12/231, 21 RAM NAGAR Ludhiana Ludhiana Punjab 141003","['{""house_number"": ""12/231"", ""street_name"": ""RAM NAGAR"", ""city"": ""Ludhiana"", ""state"": ""Punjab"", ""pincode"": ""141003""}']","['\n\n\nYou are parsing an input address (in India) into its components. The output components should be a JSON dictionary, with the key as the component...",False


41.67

# BootstrapFewShotWithRandomSearch

In [ ]:
# Accidentally spent $12 on this with `num_candidate_programs=20`, caution!

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

teleprompter = BootstrapFewShotWithRandomSearch(metric=metric1, 
                                                max_bootstrapped_demos=4,
                                                max_labeled_demos=4, 
                                                max_rounds=1,
                                                num_candidate_programs=2,
                                                num_threads=2)

# also common to init here, e.g. Rag()
second_compiled_rag = teleprompter.compile(uncompiled_rag, trainset=trainset)

In [ ]:
second_compiled_rag("What do cross encoders do?")

In [ ]:
llm.inspect_history(n=1)

In [ ]:
evaluate(second_compiled_rag, metric=metric1)

# BayesianSignatureOptimizer

In [ ]:
from dspy.teleprompt import BayesianSignatureOptimizer

llm_prompter = dspy.OpenAI(model='gpt-4', max_tokens=2000, model_type='chat')

teleprompter = BayesianSignatureOptimizer(task_model=dspy.settings.lm,
                                          metric=metric1,
                                          prompt_model=llm_prompter,
                                          n=5,
                                          verbose=False)

kwargs = dict(num_threads=1, display_progress=True, display_table=0)
third_compiled_rag = teleprompter.compile(RAG(), devset=devset,
                                         optuna_trials_num=3,
                                         max_bootstrapped_demos=4,
                                         max_labeled_demos=4,
                                         eval_kwargs=kwargs)

In [ ]:
third_compiled_rag("What do cross encoders do?")

# Check this out!!

Below you can see how the BayesianSignatureOptimizer jointly (1) optimizes the task instruction to:

```
Assess the context and answer the given questions that are predominantly about software usage, process optimization, and troubleshooting. Focus on providing accurate information related to tech or software-related queries.
```

As well as sourcing input-output examples for the prompt!

In [ ]:
llm.inspect_history(n=1)

In [ ]:
evaluate(third_compiled_rag, metric=metric1)

# Test Set Eval

In [ ]:
# Evaluate Uncompiled
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate = Evaluate(devset=testset, num_threads=1, display_progress=True, display_table=5)

In [ ]:
evaluate(uncompiled_rag, metric=metric1)

In [ ]:
evaluate(compiled_rag, metric=metric1)

In [ ]:
evaluate(second_compiled_rag, metric=metric1)

In [ ]:
evaluate(third_compiled_rag, metric=metric1)